Use this utlity to update the returns and std_dev fields within investment-options.csv

In [1]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import brownbear as bb

# Format price data.
pd.options.display.float_format = '{:0.2f}'.format

%matplotlib inline

In [3]:
# Set size of inline plots.
'''note: rcParams can't be in same cell as import matplotlib
   or %matplotlib inline
   
   %matplotlib notebook: will lead to interactive plots embedded within
   the notebook, you can zoom and resize the figure
   
   %matplotlib inline: only draw static images in the notebook
'''
plt.rcParams["figure.figsize"] = (10, 7)

Globals

In [4]:
# Set refresh_timeseries=True to download timeseries.  Otherwise /symbol-cache is used.
refresh_timeseries = True

In [5]:
# Read in investment options input csv.
df = pd.read_csv('investment-options-in.csv', skip_blank_lines=True, comment='#')
symbols = list(df['Investment Option'])
df

,Investment Option,Description,Asset Class,1 mo,3 mo,1 Yr,3 Yr,5 Yr,Vola,DS Vola,Std Dev
0,ROAD,Construction Partners Inc,US Stocks:Industrials,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,EHC,Encompass Health,US Stocks:Healthcare,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,RF,Regions Financial,US Stocks:Financials,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,VMC,Vulcan Materials Company,US Stocks:Materials,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ADTN,ADTRAN,US Stocks:Technology,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,PRA,ProAssurance Corporation,US Stocks:Financials,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,TBRG,"TruBridge, Inc.",US Stocks:Healthcare,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,MPW,"Medical Properties Trust, Inc.",US Stocks:Real Estate,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,LAKE,"Lakeland Industries, Inc.",US Stocks:Consumer Staples,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
# Eliminate repeat symbols.
symbols = set(list(symbols))

In [7]:
# Get the timeseries for the symbols and compile into a single csv.
bb.fetch_timeseries(symbols, refresh=refresh_timeseries)
bb.compile_timeseries(symbols)

MPW TBRG LAKE PRA EHC ADTN ROAD RF VMC 


In [8]:
# Read symbols timeseries into a dataframe.
df = pd.read_csv('symbols-timeseries.csv', skip_blank_lines=True, comment='#')
df.set_index("Date", inplace=True)
df

,MPW,TBRG,LAKE,PRA,EHC,ADTN,ROAD,RF,VMC
Date,,,,,,,,,
2019-01-02,9.94,25.00,10.43,36.80,44.92,9.58,9.56,10.72,92.51
2019-01-03,10.25,24.87,10.33,36.43,44.12,9.61,9.66,10.70,91.56
2019-01-04,10.21,25.42,10.67,37.99,45.20,9.94,9.55,11.10,96.25
2019-01-07,10.38,25.81,10.92,37.84,46.33,10.19,9.75,11.29,98.16
2019-01-08,10.57,26.18,11.27,37.69,47.31,10.45,10.49,11.40,97.77
...,...,...,...,...,...,...,...,...,...
2025-01-27,4.65,22.98,23.10,15.20,96.35,9.49,79.55,24.77,274.99
2025-01-28,4.45,23.60,24.25,15.26,96.78,9.63,78.80,24.75,275.10
2025-01-29,4.40,24.25,24.40,15.27,95.57,9.82,79.31,24.58,272.20


In [9]:
annual_returns = bb.annualized_returns(df, timeperiod='daily', years=1)
annual_returns

MPW     63.19
TBRG   135.26
LAKE    27.53
PRA      8.80
EHC     39.13
ADTN    61.28
ROAD    78.95
RF      32.37
VMC     19.50
dtype: float64

In [10]:
# Calculate 1 month, 3 months, 1 year, 3 year, and 5 year annualized returns.
annual_returns_1mo = bb.annualized_returns(df, timeperiod='daily', years=1/12)
annual_returns_3mo = bb.annualized_returns(df, timeperiod='daily', years=3/12)
annual_returns_1yr = bb.annualized_returns(df, timeperiod='daily', years=1)
annual_returns_3yr = bb.annualized_returns(df, timeperiod='daily', years=3)
annual_returns_5yr = bb.annualized_returns(df, timeperiod='daily', years=5)

In [11]:
# Calculate 20 day annualized volatility.
daily_returns = df.pct_change()
years = bb.TRADING_DAYS_PER_MONTH / bb.TRADING_DAYS_PER_YEAR
vola = bb.annualized_standard_deviation(daily_returns, timeperiod='daily', years=years)

In [12]:
# Calculate 20 day annualized downside volatility.
ds_vola = bb.annualized_standard_deviation(daily_returns, timeperiod='daily', years=years, downside=True)

In [13]:
# Resample df on a monthly basis.
df.index = pd.to_datetime(df.index)
monthly = df.resample('ME').ffill()

In [14]:
# Calculate monthly returns.
monthly_returns = monthly.pct_change()

In [15]:
# Calculate 3 year annualized standard deviation.
std_dev = bb.annualized_standard_deviation(monthly_returns, timeperiod='monthly', years=3)

In [16]:
# Read investment-options-in.csv
lines = []
with open('investment-options-in.csv', 'r') as f:
    lines = [line.strip() for line in f]
#lines

In [17]:
# For each etf, write out the 1 Yr, 3 Yr, 5 Yr, and std dev.
out = []
for line in lines:
    # Copy empty and comment lines.
    if not line or line.startswith('#'):
        out.append(line)
        continue
    # Split line by comma delimiter; strip the fields.
    fields = line.split(',')
    fields = [field.strip() for field in fields]
    symbol = fields[0].strip('\"')
    # Copy header.
    if symbol == 'Investment Option':
        out.append(line)
        continue
    ret_1mo = annual_returns_1mo[symbol]
    ret_3mo = annual_returns_3mo[symbol]
    ret_1yr = annual_returns_1yr[symbol]
    ret_3yr = annual_returns_3yr[symbol]
    ret_5yr = annual_returns_5yr[symbol]
    
    if np.isnan(ret_3yr): ret_3yr = ret_1yr
    if np.isnan(ret_5yr): ret_5yr = ret_3yr

    _vola = vola[symbol]*100
    _ds_vola = ds_vola[symbol]*100
    sd = std_dev[symbol]*100

    out.append('{},{},{},"{:0.2f}","{:0.2f}","{:0.2f}","{:0.2f}","{:0.2f}","{:0.2f}","{:0.2f}","{:0.2f}"'
               .format(fields[0], fields[1], fields[2],
                       ret_1mo, ret_3mo, ret_1yr, ret_3yr, ret_5yr, _vola, _ds_vola, sd)) 

In [18]:
# Write out asset-classes.csv
with open('investment-options.csv', 'w') as f:
    for line in out:
        f.write(line + '\n')